In [14]:
import pandas as pd
import seaborn as sns

from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from label_map import label2id, id2label

## 1 뉴스 본문 전처리, 요약, 종목명, 업종명, 임베딩

### 1 뉴스 본문 전처리

In [15]:
df = pd.read_csv("../../db/news_2023_2025.csv")

In [16]:
df.head(2)

,news_id,wdate,title,article,press,url,image
0,20250523_0001,2025-05-23 19:11,[마켓인]모태펀드 존속 불확실성 해소될까…이재명 공약에 업계 주목,"2035년 종료 앞둬, 존속 공약에 기대감\n창업 초기자금 공백 완화 가능성에 업계...",이데일리,https://n.news.naver.com/mnews/article/018/000...,https://imgnews.pstatic.net/image/018/2025/05/...
1,20250523_0002,2025-05-23 18:52,"[단독] 카카오페이, 2500만 회원 쓱·스마일페이 품나…간편결제 시장 빅3 경쟁 후끈",매각가 5000억 안팎 달할듯\n결제시장 내 입지강화 포석\n카카오페이 [사진 = ...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...


In [24]:
import re
from kss import split_sentences

def remove_market_related_sentences(text: str) -> str:
    # 줄바꿈 제거
    text = text.replace("\n", " ")

    # 대괄호 포함 텍스트 제거: [파이낸셜뉴스], [사진] 등
    text = re.sub(r"\[[^\]]*\]", "", text)

    # '/사진', '/사진제공' 제거
    text = re.sub(r"/사진(제공)?", "", text)

    # 이메일 주소 제거 (예: josh@yna.co.kr)
    text = re.sub(r"\b[\w.-]+@[\w.-]+\.\w+\b", "", text)

    # 문장 단위 분리 (간단하게 마침표 기준, 필요시 KSS 등 적용 가능)
    sentences = split_sentences(text)

    # 제거할 패턴들 (뉴스 문장에서 자주 등장하는 패턴)
    patterns = [
        r"(자세한 내용|자세한 사항)",  # 뉴스 기본 표현
        r"\d{4}[.-]\d{1,2}[.-]\d{1,2}",  # 날짜 (예: 2025.03.26, 2024-12-01)
        r"([0-9,]+(?:만)?[0-9,]*\s?(?:원|만원))",  # 가격 (예: 3,500원, 12000원)
        r"(강세|펀드|시가총액|등락률|한국거래소)",  # 증시 용어
        r"\([+-]?[0-9.,]+%\)",  # 괄호 안 퍼센트 등락률
        r"(투자의견|연구원|평가|예상치|증권가|리포트|팀장)",  # 애널리스트 용어
        r"(순이익|전년|매출|영업이익|영업적자|증시|코스피|코스닥|다우|나스닥|매출액|거래일|호조세|레버리지|투자자|조정|자산|수익률|이익률|수익성|내리막|부진한|낙폭|기대치|실적발표|기업 가치)",  # 시장 용어
    ]

    # 하나의 통합 패턴으로 컴파일
    combined_pattern = re.compile("|".join(patterns))

    # 필터링된 문장만 유지
    filtered = [s for s in sentences if not combined_pattern.search(s)]

    text_preprocessed = " ".join(filtered)

    # print(f"원문:{sentences}\n|\n전처리 된 문장: {text_preprocessed}\n\n")

    return text_preprocessed

In [25]:
from tqdm import tqdm

tqdm.pandas()  # 이거 반드시 호출해야 함

df["article_preprocessed"] = df[
    "article"
].progress_apply(remove_market_related_sentences)

100%|██████████| 58405/58405 [28:07<00:00, 34.61it/s]  


In [30]:
def drop_article_preprocessed(df):
    # 전처리된 문장 길이 구하기
    df["article_preprocessed_len"] = df[
        "article_preprocessed"
    ].apply(lambda x: len(x))

    # 전처리된 문장 길이가 70 이상인 것만 필터링
    df = df[
        df["article_preprocessed_len"] > 70
    ]

    df = df.drop(columns=["article_preprocessed_len"])

    return df

In [31]:
df = drop_article_preprocessed(df)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53202 entries, 0 to 58404
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   news_id               53202 non-null  object
 1   wdate                 53202 non-null  object
 2   title                 53202 non-null  object
 3   article               53202 non-null  object
 4   press                 53202 non-null  object
 5   url                   53202 non-null  object
 6   image                 53202 non-null  object
 7   article_preprocessed  53202 non-null  object
dtypes: object(8)
memory usage: 3.7+ MB


### 2 뉴스 본문 요약

In [20]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 모델 이름
model_summarize_name1 = "digit82/kobart-summarization"

# 토크나이저 & 모델 로드
tokenizer_summarize1 = AutoTokenizer.from_pretrained(model_summarize_name1)
model_summarize1 = AutoModelForSeq2SeqLM.from_pretrained(
    model_summarize_name1, use_safetensors=True
)

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_summarize1.to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [33]:
def summarize_event_focused(text):
    # 토크나이징 및 텐서 변환 (GPU로 올리기)
    inputs = tokenizer_summarize1(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    input_ids = inputs["input_ids"].to(device)

    text_length = len(text)

    # 1. 최소 길이는 짧은 본문은 고정, 긴 본문은 점진적으로 증가
    def compute_min_length(text_length: int) -> int:
        if text_length < 300:
            return 30
        else:
            return 50

    min_len = compute_min_length(text_length)
    max_len = round(text_length * 0.5) + 50  # 더 여유를 주되 max 길이 제한

    # 2. generate 최적 설정
    summary_ids = model_summarize1.generate(
        input_ids,
        min_length=min_len,
        max_new_tokens=max_len,
        num_beams=4,  # 4보다 빠름. 품질도 비슷
        length_penalty=1.0,  # 길이 패널티 완화
        repetition_penalty=1.3,  # 반복 억제 강화
        no_repeat_ngram_size=3,  # 반복 문장 방지
        early_stopping=True,
        do_sample=False,  # 일관된 요약
    )

    return tokenizer_summarize1.decode(summary_ids[0], skip_special_tokens=True)


def print_summary(df, max_tokens=128):
    for i, row in tqdm(df.iterrows(), total=len(df)):
        try:
            text = row["article_preprocessed"]
            summary = summarize_event_focused(text)
            print(f"\n📝 뉴스 {i}\n\n본문: {text} \n\n요약: {summary}\n")
        except Exception as e:
            print(f"❌ 요약 실패: {e}")

In [34]:
print_summary(df[:3])

 33%|███▎      | 1/3 [00:04<00:09,  4.57s/it]


📝 뉴스 0

본문: 2035년 종료 앞둬, 존속 공약에 기대감 창업 초기자금 공백 완화 가능성에 업계 안도 VC, "정책 연속성 중요…불확실성 줄어야" 퇴직연금도 벤처로…BDC 등 활성화 방안 포함 이 기사는 2025년05월23일 17시10분에 마켓인 프리미엄 콘텐츠 로 선공개 되었습니다. 23일 관련업계에 따르면 이 후보는 해당 공약을 ‘벤처·스타트업 10대 공약’의 하나로 포함시키며, 벤처 투자 인프라를 제도적으로 보완하겠다는 의지를 내비쳤다. 업계에서는 단기 예산 복원이 아닌, 정책 연속성에 무게를 둔 점에 주목하고 있다. 글로벌 대비 민간 자본의 투자 여력이 제한적인 한국 벤처 시장 특성상, 정부 출자의 존재감은 그만큼 절대적이다. 이에 따라 벤처캐피털(VC) 업계에서는 이재명 후보의 공약이 단순한 예산 복원을 넘어 구조적인 불확실성 해소에 방점을 두고 있다는 점에서 긍정적으로 받아들이는 분위기다. 이 후보는 이외에도 퇴직연금의 벤처투자 허용, 연기금 투자풀의 벤처투자 확대, 기업 성장집합투자기구(BDC) 도입 등 다양한 벤처투자 활성화 방안을 함께 제시하며 투자 생태계 전반의 구조 개편 의지를 밝혔다. 

요약: 이재명 후보의 벤처투자 허용, 연기금 투자풀의 벤처투자 확대, 기업 성장집합투자기구(BDC) 도입 등 다양한 벤처투자 활성화 방안을 함께 제시하며 투자 생태계 전반의 구조 개편 의지를 내비쳤다.   



 67%|██████▋   | 2/3 [00:06<00:02,  2.82s/it]


📝 뉴스 1

본문: 매각가 5000억 안팎 달할듯 결제시장 내 입지강화 포석 카카오페이  국내 대표 전자결제사업자인 카카오페이가 신세계이마트 산하 간편결제사업부 인수에 나섰다. 네이버페이·토스페이에 대항해 시장 점유율을 늘리려는 포석으로 해석된다. 23일 정보기술(IT)·투자은행(IB) 업계에 따르면 카카오페이가 SSG닷컴 쓱페이와 G마켓 스마일페이 인수를 위해 신세계이마트 측과 협상을 진행 중인 것으로 파악됐다. 업계에서는 매각가가 5000억원 안팎에 달할 것으로 전망했다. 카카오페이는 약 2500만명의 이용자를 보유한 쓱페이·스마일페이를 인수할 경우 사업 외연을 확대할 수 있다. 매각 작업에 정통한 업계 고위 관계자는 “카카오페이가 결제시장 내 입지 강화를 위해 신세계이마트와 결제사업 부문 인수 등 다양한 방안에 대해 논의 중인 것으로 안다”고 전했다. 신세계이마트 측은 “매각을 재추진중인 건 맞지만 아직 정해진 것은 없다”고 밝혔다. 신세계이마트는 지난 해 토스와 결제사업 매각을 논의하다 결렬된 바 있다. 

요약: 23일 정보기술(IT)·투자은행(IB) 업계에 따르면 국내 대표 전자결제사업자인 카카오페이가 SSG닷컴 쓱페이와 G마켓 스마일페이 인수를 위해 신세계이마트 측과 협상을 진행 중인 것으로 알려졌다.



100%|██████████| 3/3 [00:07<00:00,  2.35s/it]


📝 뉴스 2

본문: 총 2166대1의 경쟁률을 기록, 청약 증거금 6조원을 모으는 데 성공했다. 지난 22일부터 이날까지 양간 일반 청약을 진행한 결과 최종 경쟁률은 총 2166.01대1로  집계됐다. 청약 건수는 19만1049건으로, 청약 금액의 절반을 미리 내는 청약증거금은 약 6조1400억원이 모였다. 키스트론은 고려제강그룹 계열사로 1992년 설립됐다. 철선에 구리를 도금한 동복강선 리드 와이어가 핵심 제품으로, 글로벌 시장에서 약 18% 점유율을 갖췄다. 이번 공모를 통해 226억 원을 조달한다. 주관사는 신한투자증권이 맡았다. 

요약: 고려제강그룹 계열사로 1992년 설립된 키스트론은 지난 22일부터 양간 일반 청약을 진행한 결과 최종 경쟁률은 총 2166.01대1로 집계됐다.



In [37]:
df = df[:20000]

In [ ]:
df["summary"] = df[
    "article_preprocessed"
].progress_apply(summarize_event_focused)

  0%|          | 49/20000 [01:11<8:19:21,  1.50s/it] 

### 3 뉴스 종목 매칭하기

In [21]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from label_map import label2id, id2label

model_name = "KPF/KPF-BERT-NER"

# 토크나이저와 모델 불러오기
tokenizer_ner = AutoTokenizer.from_pretrained(model_name)
model_ner = AutoModelForTokenClassification.from_pretrained(
    model_name, use_safetensors=True
)

# 라벨 매핑 반영
model_ner.config.label2id = label2id
model_ner.config.id2label = id2label

tokenizer_ner.model_max_length = 512

# 디바이스 설정
device = 0 if torch.cuda.is_available() else -1

# NER 파이프라인 생성 (GPU 사용)
ner_pipeline = pipeline(
    task="ner",
    model=model_ner,
    tokenizer=tokenizer_ner,
    aggregation_strategy="simple",
    framework="pt",
    device=device,
)

Device set to use cpu


In [ ]:
def extract_ner(text):
    entities = ner_pipeline(text)
    results = []
    seen = set()

    for ent in entities:
        word = ent["word"].replace("##", "").strip()
        tag = ent["entity_group"]

        score = ent["score"]

        if word and score >= 0.95 and (word, tag) not in seen:
            results.append((word, tag))
            seen.add((word, tag))

    return results


def get_stock_names(text):
    ner_list = extract_ner(text)

    # OGG_ECONOMY만 필터링하여 종목명만 리스트로 추출
    stock_names = [ent[0] for ent in ner_list if ent[1] == "OGG_ECONOMY"]

    return stock_names

In [ ]:
df["stock_list"] = df["summary"].progress_apply(
    get_stock_names
)

In [ ]:
# 종목명이 1개 이상, 4개 이하인 뉴스만 필터링하기

df["stock_list_len"] = df["stock_list"].apply(lambda x: len(x))

df = df[(df["stock_list_len"] >= 1) & (df["stock_list_len"] <= 4)]

df = df.drop(columns=["stock_list_len"])

### 4 뉴스 업종명 매칭하기

In [ ]:
df_krx_desc = pd.read_csv("../../db/kospi_description.csv", encoding="cp949")
df_krx_desc.head()

In [ ]:
# 종목명 → 업종명 매핑 딕셔너리 생성
stock_to_industry = dict(zip(df_krx_desc["종목명"], df_krx_desc["업종명"]))

# 업종명을 리스트로 매핑
def get_industry_list(stock_list):
    return [
        stock_to_industry.get(stock, "")
        for stock in stock_list
        if stock_to_industry.get(stock, "") != ""
    ]

# 새로운 컬럼에 저장
df["industry_list"] = df["stock_list"].apply(get_industry_list)

In [ ]:
# 업종명이 1개 이상인 뉴스만 필터링하기

df["industry_list_len"] = df["industry_list"].apply(lambda x: len(x))

df = df[df["industry_list_len"] >= 1]

df = df.drop(columns=["industry_list_len"])

### 5 뉴스 요약문 임베딩하기

In [22]:
from sentence_transformers import SentenceTransformer
import torch

model_name = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"

# GPU가 있으면 GPU, 없으면 CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# 모델 로딩 후 디바이스 설정
model_emb = SentenceTransformer(model_name, device=device)

In [ ]:
embeddings = model_emb.encode(
    df["summary"].tolist(), show_progress_bar=True
)

df["summary_embedding"] = embeddings.tolist()

In [ ]:
df.to_csv("../../db/news_2023_2025_summarized.csv")

## 2 뉴스 경제 및 행동 지표 피쳐 추가
- 주가 D+1~D+30 변동률, 금리, 환율, 기관 매매동향, 유가 등

## 3 뉴스 시멘틱 피쳐 추가
-  topic별 분포값, 클러스터 동일 여부

## 4 뉴스 관계 피쳐 추가
- cosine 유사도, 동일 종목, 동일 키워드 여부 등